# Demo for LumiSpy package working with AttoLight SEM CL data

This notebook shows:

- Loading a `HYPCard.bin` file correctly with the lumispy load function
- Plotting cathodoluminescence data in different ways
- Dealing with metadata
- Correcting for spectral defects

Import packages:

In [1]:
%matplotlib qt 
#%matplotlib inline
import lumispy as lum
import hyperspy.api as hs

import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import numpy as np
import addcopyfighandler

### Loading luminescence files

In [2]:
cl_sem = hs.load('data/cathodoluminescence_example.hspy')

### Plotting data

Plot the corrected data:

In [10]:
cl_sem.plot()

### Panchromatic image:

In [4]:
cl_sem.T.mean().plot(cmap='viridis')

In [5]:
cl_sem.mean().plot()

# Get plotting

## Get the colour filtered panchromatic images

Select two energy regions to plot as coloured images (same way it is done in the AttoLight software)

In [6]:
im = cl_sem.T
im.plot()
roi1 = hs.roi.SpanROI(left=650, right=700)      #sets a digitalbandfilter
im_roi1 = roi1.interactive(im, color="green")

In [7]:
roi1

SpanROI(left=650, right=700)

In [8]:
fig = plt.figure(figsize=(3.5,5),)
gridsize = (1, 1)
plt.matplotlib.gridspec.GridSpec(gridsize[0], gridsize[1])

ax1 = plt.subplot2grid(gridsize, (0,0))
ax1.imshow(roi1(im).mean(), cmap='Oranges')
roi_width = roi1.right - roi1.left
roi_centre = roi1.left + 0.5*roi_width
ax1.set_title("{:.0f} $\pm$ {:.0f} nm".format(roi_centre, roi_width/2), color='orange')
plt.axis('off')

units = roi1(im).mean().axes_manager[0].units
scale = roi1(im).mean().axes_manager[0].scale
scalebar = ScaleBar(scale, units, location='lower right') # 1 pixel = 0.2 meter
plt.gca().add_artist(scalebar)
plt.suptitle(roi1(im).mean().metadata.General.title, y=0.03)
plt.axis('off')
plt.tight_layout(rect=(0,0.03,1,1))

## Fitting Gaussian

Select the starting parameters

In [9]:
cl_sem.plot()

In [10]:
####################################
#MODEL
g1_centre = 513   # Guess for centre wavelength
g1_max = g1_centre + 20      # Max value for centre wavelength
g1_min = g1_centre - 20      # Min value for centre wavelength

g1_fwhm = 50            #Guess for FWHM
g1_fwhm_max = 150       #Maxvalue for FWHM
g1_fwhm_min = 1         #Minvalue for FWHM

g1_h = 15           #Guess for peak Intensity
g1_h_max = 40      #Maxvalue for peak Intesity
g1_h_min = 0         #Minvalue for peak Intensity

bkg_offset = 650  #Background to be substracted

In [11]:
m = cl_sem.create_model()

#Background offset
bkg = hs.model.components1D.Offset()
#Gaussian peak
g1 = hs.model.components1D.Expression(
expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
name="Perovskite Peak",
position="x0",
height=1,
fwhm=1,
x0=0,
module="numpy")
#Tweak guessed initial parameters
m.extend([g1, bkg])
g1.x0.value, g1.x0.bmax, g1.x0.bmin = g1_centre, g1_max, g1_min
g1.fwhm.value, g1.fwhm.bmax, g1.fwhm.bmin = g1_fwhm, g1_fwhm_max, g1_fwhm_min
g1.height.value, g1.height.bmax, g1.height.bmin = g1_h, g1_h_max, g1_h_min
bkg.offset.value = bkg_offset

In [12]:
#Fit for all the positions
m.multifit(bounded=True, show_progressbar=True)

KeyboardInterrupt: 

In [ ]:
m.print_current_values()

In [ ]:
#Plot the fit on the raw data
m.plot(plot_components=True)

In [ ]:
m_x0 = g1.x0.as_signal()
m_x0.plot(cmap='inferno')
m_intensity = g1.height.as_signal()
m_intensity.plot(cmap='viridis')

You can do particle segmentation using model fitting:

In [ ]:
#Make mask to remove region where the intensity is below the mean value:
mask_treshold = m_intensity.data.mean()
mask = m_intensity.data > mask_treshold #Returns a boolean matrix mask
plt.imshow(mask)

In [ ]:
m_x0 = g1.x0.as_signal()
m_x0.data[~mask] = np.nan #This replaces all of values in m_x0.data that correspond to False values in the mask with np.nan
m_x0.plot(cmap='viridis')